# ⚙️ Line Balancing Simulation
Methods: Largest Candidate Rule (LCR) and Ranked Positional Weight (RPW)

In [ ]:
import pandas as pd

# Example task data
tasks = pd.DataFrame({
    'Task': list('ABCDEFG'),
    'Time': [5, 7, 3, 2, 4, 6, 5],
    'Predecessors': [None, 'A', 'A', 'B', 'B', 'C', 'D']
})

cycle_time = 10  # minutes

print('--- Task Data ---')
print(tasks)

## Largest Candidate Rule (LCR)

In [ ]:
tasks_sorted = tasks.sort_values(by='Time', ascending=False).reset_index(drop=True)
stations = []
current_station = []
current_time = 0

for _, row in tasks_sorted.iterrows():
    if current_time + row['Time'] <= cycle_time:
        current_station.append(row['Task'])
        current_time += row['Time']
    else:
        stations.append(current_station)
        current_station = [row['Task']]
        current_time = row['Time']

stations.append(current_station)
print('\n--- LCR Workstation Assignments ---')
for i, s in enumerate(stations, 1):
    print(f"Station {i}: {s}")

## Ranked Positional Weight (RPW)

In [ ]:
# Compute positional weights
def get_successors(task, preds):
    return [t for t, p in preds.items() if p == task]

preds = dict(zip(tasks['Task'], tasks['Predecessors']))
positional_weight = {}

for task in tasks['Task']:
    total_time = tasks.loc[tasks['Task'] == task, 'Time'].values[0]
    to_check = [task]
    while to_check:
        current = to_check.pop()
        succ = get_successors(current, preds)
        for s in succ:
            total_time += tasks.loc[tasks['Task'] == s, 'Time'].values[0]
            to_check.append(s)
    positional_weight[task] = total_time

tasks['PositionalWeight'] = tasks['Task'].map(positional_weight)
tasks_rpw = tasks.sort_values(by='PositionalWeight', ascending=False)

print('\n--- RPW Order ---')
print(tasks_rpw[['Task', 'PositionalWeight']])